In [1]:
import pandas as pd
import numpy as np
import configparser
from ast import literal_eval
import ast

In [2]:
behaviors = pd.read_csv('behaviors_covid_ids_deID.csv', index_col = 0)

In [3]:
behaviors['Date'] = pd.to_datetime(behaviors['Date'])

In [4]:
# Filter out midnight cases
# we have some that are the correct time, can reassign these manually later.
print(behaviors['Time'].unique())
behaviors = behaviors[behaviors['Time'] != '00:00:00']
print(behaviors['Time'].unique())

# We want to get dates of behavior and 3 shifts per day at least.
# Basically we want to create a blank date and time thing, and then concatenate those
# then we want to concatenate the one on the behavior file, and merge both on that by person/date/time
# Will have to be a left join
behaviors['Date'] = pd.to_datetime(behaviors['Date'],format = "%m/%d/%Y %I:%M:%S %p")
#behaviors['Time'] = pd.to_datetime(behaviors['Time'])

# Create concat column
behaviors['Concat'] = behaviors['Person_ID'].astype(str) + " " + behaviors['Date'].astype(str)

print(behaviors)
behaviors_grouped = behaviors[['Person_ID','Date']].sort_values('Date').groupby('Person_ID').agg(['first','last'])

behaviors_grouped.columns = behaviors_grouped.columns.get_level_values(1)
behaviors_grouped = behaviors_grouped.reset_index().copy()


df = behaviors_grouped.copy()
df['Dates'] = [pd.date_range(x, y) for x , y in zip(df['first'],df['last'])]
df = df.explode('Dates')


df = df.drop(['first','last'],axis=1)


df['Times'] = "['07:00:00','15:00:00', '23:00:00']"
df['Times'] = df['Times'].apply(ast.literal_eval)
df = df.explode('Times').copy()

behaviors['Concat'] = behaviors['Person_ID'].astype(str) + " " + behaviors['Date'].astype(str) + " " + behaviors['Time'].astype(str)


df['Concat'] = df['Person_ID'].astype(str) + " " + df['Dates'].astype(str) + " " + df['Times'].astype(str)
df['Concat_Date'] = df['Person_ID'].astype(str) + " " + df['Dates'].astype(str)
df = df[['Concat','Concat_Date']].copy()


behaviors = df.merge(behaviors, on = 'Concat', how = 'left')
behaviors.columns
behaviors


['23:00:00' '15:00:00' '07:00:00' '00:00:00']
['23:00:00' '15:00:00' '07:00:00']
            Person_ID       Date      Time         Target  \
Unnamed: 0                                                  
838118              1 2019-09-15  23:00:00            sib   
2140342             2 2019-09-15  15:00:00     disruptive   
838121              1 2019-09-15  23:00:00     disruptive   
838120              1 2019-09-15  23:00:00  mouthing pica   
838119              1 2019-09-15  23:00:00     aggression   
...               ...        ...       ...            ...   
5097602            31 2020-09-08  15:00:00            NaN   
3249998            39 2020-09-08  07:00:00     aggression   
3249999            39 2020-09-08  07:00:00            sib   
3250000            39 2020-09-08  07:00:00     disruptive   
3250001            39 2020-09-08  07:00:00            sib   

            Time_Sample_Percent Behavior_No_Data_Recorded  Episode_Count  \
Unnamed: 0                                       

,Concat,Concat_Date,Person_ID,Date,Time,Target,Time_Sample_Percent,Behavior_No_Data_Recorded,Episode_Count,Behavior_LOA,...,Duration_03_Count,Duration_04_Count,Duration_05_Count,Duration_06_Count,Intensity_01_Count,Intensity_02_Count,Intensity_03_Count,Intensity_04_Count,Intensity_05_Count,FirstDate
0,1 2019-09-15 07:00:00,1 2019-09-15,1.0,2019-09-15,07:00:00,NaN,NaN,False,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01
1,1 2019-09-15 15:00:00,1 2019-09-15,1.0,2019-09-15,15:00:00,NaN,NaN,False,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01
2,1 2019-09-15 23:00:00,1 2019-09-15,1.0,2019-09-15,23:00:00,sib,0.0,False,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01
3,1 2019-09-15 23:00:00,1 2019-09-15,1.0,2019-09-15,23:00:00,disruptive,0.0,False,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01
4,1 2019-09-15 23:00:00,1 2019-09-15,1.0,2019-09-15,23:00:00,mouthing pica,0.0,False,0.0,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253565,78 2020-05-12 15:00:00,78 2020-05-12,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253566,78 2020-05-12 23:00:00,78 2020-05-12,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253567,78 2020-05-13 07:00:00,78 2020-05-13,78.0,2020-05-13,07:00:00,aggression,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
253568,78 2020-05-13 15:00:00,78 2020-05-13,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Indicate places where the shift does not exist - was never recorded
behaviors['Missing_Shift'] = 0
behaviors.loc[behaviors['Person_ID'].isna(),'Missing_Shift'] = 1

In [6]:
# Convert all behavior columns to float type
# and confirm type
cols = [col for col in behaviors.columns if 'Behavior' in col]
behaviors[cols] = behaviors[cols].astype('float64') 
behaviors.loc[:,behaviors.columns.str.contains('Behavior')].dtypes

Behavior_No_Data_Recorded    float64
Behavior_LOA                 float64
Behavior_None                float64
dtype: object

In [7]:
# Sum columns of duration and intensity counts across
behaviors['Duration_Count'] = behaviors.loc[:, behaviors.columns.str.match('^Duration.*Count$')].sum(axis=1,min_count=1)
behaviors['Intensity_Count'] = behaviors.loc[:, behaviors.columns.str.match('^Intensity.*Count$')].sum(axis=1,min_count=1)

# Drop extraneous duration and intensity columns since they've been summed
behaviors = behaviors.drop(behaviors.columns[behaviors.columns.str.match('^Duration_.*_Count$').tolist()],axis=1)
behaviors = behaviors.drop(behaviors.columns[behaviors.columns.str.match('^Intensity_.*_Count$').tolist()],axis=1)

# Time sampling is a different way of recording behavior than counting episodes, 
# so we exclude those that are recorded this way
behaviors.loc[behaviors['Time_Sample_Percent'] > 0,'Behavior_No_Data_Recorded'] = 1.0

# If there are any non-integer episodes, they probably are some sort of time sample too
behaviors.loc[pd.notna(behaviors['Episode_Count']) & (behaviors['Episode_Count'] % 1 != 0),'Behavior_No_Data_Recorded'] = 1.0

# Where are duration count OR intensity count not null?
# If they're both not null and unequal, set equal to the max of them both
test = behaviors.loc[behaviors['Duration_Count'].notna() & behaviors['Intensity_Count'].notna() & 
          (behaviors['Duration_Count'] != behaviors['Intensity_Count']),['Duration_Count','Intensity_Count']].max(axis=1)
behaviors.loc[behaviors['Duration_Count'].notna() & behaviors['Intensity_Count'].notna() & 
          (behaviors['Duration_Count'] != behaviors['Intensity_Count']),['Duration_Count','Intensity_Count']] = test

# Aggregate duration and intensity counts into one count variable
behaviors['Count'] = behaviors['Duration_Count'].fillna(behaviors['Intensity_Count'])

# Behaviors where episode count is 0 or null should be overridden by the duration/intensity counts
behaviors.loc[((behaviors['Episode_Count'].isna() | (behaviors['Episode_Count']==0)) 
           & behaviors['Count'] > 0),'Episode_Count'] = behaviors.loc[((behaviors['Episode_Count'].isna() | 
                                                                    (behaviors['Episode_Count']==0)) & 
                                                                   behaviors['Count'] > 0),'Count']
# Drop unnecessary count columns
behaviors = behaviors.drop(['Duration_Count','Intensity_Count','Count'], axis = 1)


# Filter for where all the behavior stuff is null
# But Episode Count is 0 = there is no behavior though it was recorded
behaviors.loc[((behaviors['Behavior_None'].isna()) &
           ( behaviors['Behavior_LOA'].isna())
           & (behaviors['Behavior_No_Data_Recorded'].isna())
          & (behaviors['Episode_Count']==0)), 'Behavior_None'] = 1.0

# We can't confirm for now if there was an LOA or no data recorded, so we leave that for now.

# Depending on if behavior is aggression, sib or both, make a behavior column?
config = configparser.ConfigParser()
config.read('targets_of_interest.ini')
covid = True
if covid:
    targets = literal_eval(config['section1']['targets_of_interest'])
else:
    targets = ['aggression','sib']#

final_targets = {}
for target in targets:
    final_targets[target] = target

# Convert behaviors of aggression and SIB to their generic target dictionary types
for key in final_targets:
    current_list = [final_targets[key]]
    
    behaviors[key] = ''
    behaviors.loc[behaviors['Target'].isin(current_list),key] = key

behaviors['other'] = ''
behaviors.loc[((~behaviors['Target'].isna()) &(~behaviors['Target'].isin(targets))), 'other'] = 'other'

In [8]:
len(targets)
targets = targets + ['other']

In [9]:

cols = []
if len(targets) > 1:
    behaviors['Behavior_Episodes'] = np.where(behaviors[targets].ne('').any(axis=1) & (behaviors['Episode_Count'] > 0), behaviors['Episode_Count'], 0)
            # multiple targets
    
    for target in targets:
        col = 'Behavior_Episodes_' + target
        behaviors[col] = np.where((behaviors[target] != '') & (behaviors['Episode_Count'] > 0), behaviors['Episode_Count'], 0)
        behaviors['Behavior_No_Data_Recorded_'+target] = np.where((behaviors[target] != '') & (behaviors['Episode_Count'].isna()), 1.0, behaviors['Behavior_No_Data_Recorded']) 
        cols.append(col)


        for target in targets:
            col = 'Behavior_' + target
            behaviors[col] = np.where((behaviors[target] != '') & (behaviors['Episode_Count'] > 0), behaviors['Episode_Count'] > 0, 0)
            cols.append(col)
            col = 'Behavior_No_Data_Recorded_' + target
            behaviors[col] = np.where((behaviors[target] != '') & (behaviors['Behavior_No_Data_Recorded'] > 0), behaviors['Behavior_No_Data_Recorded'], 0)

    behaviors['Behavior'] = np.where(behaviors[targets].ne('').any(axis=1) & (behaviors['Episode_Count'] > 0), behaviors['Episode_Count'] > 0, 0)
   

In [10]:
# Note that each is one observation before grouping by shift
behaviors['n_observations'] = 1

In [11]:
keys = ['n_observations','Missing_Shift'] + ['Behavior_' + target for target in targets] + ['Behavior_Episodes_' + target for target in targets] + ['Behavior_LOA',
                                                                                                               'Behavior_None','Behavior_No_Data_Recorded'] + ['Behavior_No_Data_Recorded_'+ target for target in targets]
values = [ 'sum']

behaviors_wide = behaviors.copy()

kvs = {k:values for k in keys }

behaviors_wide = behaviors_wide.groupby('Concat').agg(kvs)

In [12]:
behaviors_wide.columns = behaviors_wide.columns.get_level_values(0)
behaviors_wide.columns

Index(['n_observations', 'Missing_Shift', 'Behavior_aggression',
       'Behavior_sib', 'Behavior_elopement', 'Behavior_disruptive',
       'Behavior_other', 'Behavior_Episodes_aggression',
       'Behavior_Episodes_sib', 'Behavior_Episodes_elopement',
       'Behavior_Episodes_disruptive', 'Behavior_Episodes_other',
       'Behavior_LOA', 'Behavior_None', 'Behavior_No_Data_Recorded',
       'Behavior_No_Data_Recorded_aggression', 'Behavior_No_Data_Recorded_sib',
       'Behavior_No_Data_Recorded_elopement',
       'Behavior_No_Data_Recorded_disruptive',
       'Behavior_No_Data_Recorded_other'],
      dtype='object')

In [13]:
# Where there's only one thing recorded and it's something that we don't care about, we say it's not recorded
behaviors_wide.loc[(behaviors_wide['n_observations']==1) & (behaviors_wide['Behavior_No_Data_Recorded_other']>0),'Behavior_No_Data_Recorded'] = 1.0

# If it's missing the shift altogether, say no data is recorded
behaviors_wide.loc[behaviors_wide['Missing_Shift']==1,'Behavior_No_Data_Recorded'] = 1.0

In [14]:
behavior_cols = [col for col in behaviors_wide.columns if 'Behavior' in col]
behavior_cols = [col for col in behavior_cols if 'Episode' not in col]
behavior_cols

['Behavior_aggression',
 'Behavior_sib',
 'Behavior_elopement',
 'Behavior_disruptive',
 'Behavior_other',
 'Behavior_LOA',
 'Behavior_None',
 'Behavior_No_Data_Recorded',
 'Behavior_No_Data_Recorded_aggression',
 'Behavior_No_Data_Recorded_sib',
 'Behavior_No_Data_Recorded_elopement',
 'Behavior_No_Data_Recorded_disruptive',
 'Behavior_No_Data_Recorded_other']

In [15]:
behaviors_wide.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77793 entries, 1 2019-09-15 07:00:00 to 9 2020-08-31 23:00:00
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   n_observations                        77793 non-null  int64  
 1   Missing_Shift                         77793 non-null  int64  
 2   Behavior_aggression                   77793 non-null  int32  
 3   Behavior_sib                          77793 non-null  int32  
 4   Behavior_elopement                    77793 non-null  int32  
 5   Behavior_disruptive                   77793 non-null  int32  
 6   Behavior_other                        77793 non-null  int32  
 7   Behavior_Episodes_aggression          77793 non-null  float64
 8   Behavior_Episodes_sib                 77793 non-null  float64
 9   Behavior_Episodes_elopement           77793 non-null  float64
 10  Behavior_Episodes_disruptive          77793 non-nul

In [16]:
behaviors_wide.reset_index().to_csv('behaviors_wide_deID.csv')